# <a id='kernel-approximation'></a> 6.7. [**Approximation de noyaux**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#kernel-approximation)</br>([_Kernel Approximation_](https://scikit-learn.org/stable/modules/kernel_approximation.html#kernel-approximation))

Ce sous-module contient des fonctions qui approximent les mappings de caractéristiques correspondant à certains noyaux, tels qu'ils sont utilisés par exemple dans les machines à vecteurs de support (voir [**Machines à vecteurs de support** (1.4)](https://scikit-learn.org/stable/modules/svm.html#svm)). Les fonctions de caractéristiques suivantes effectuent des transformations non linéaires de l'entrée, qui peuvent servir de base pour la classification linéaire ou d'autres algorithmes.

L'avantage d'utiliser des mappings de caractéristiques explicites approximatifs par rapport à la [**technique des noyaux**](https://en.wikipedia.org/wiki/Kernel_trick), qui utilise les mappings de caractéristiques de manière implicite, est que les mappings explicites peuvent être mieux adaptés à l'apprentissage en ligne et peuvent réduire considérablement le coût de l'apprentissage avec des ensembles de données très volumineux. Les SVM à noyau standard ne sont pas adaptés aux ensembles de données volumineux, mais en utilisant une approximation du mapping de noyau, il est possible d'utiliser des SVM linéaires beaucoup plus efficaces. En particulier, la combinaison d'approximations de mappings de noyaux avec [**`SGDClassifier`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier) permet d'effectuer un apprentissage non linéaire sur des ensembles de données volumineux.

Comme il n'y a pas eu beaucoup de travaux empiriques utilisant des plongements approximatifs, il est conseillé de comparer les résultats avec des méthodes de noyaux exactes lorsque cela est possible.

**Voir aussi :** [**Régression polynomiale : étendre les modèles linéaires avec des fonctions de base** (1.1)](https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression) pour une transformation polynomiale exacte.

## Plan

✔  6.7. [**Approximation du noyau**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#kernel-approximation)
([*Kernel Approximation*](https://scikit-learn.org/stable/modules/kernel_approximation.html#kernel-approximation))
* **Volume** : 6 pages, 2 exemples, 7 papiers
* ✔ 6.7.1. [**Méthode Nystroem pour l'approximation du noyau**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#nystroem-method-for-kernel-approximation)
([_Nystroem Method for Kernel Approximation_](https://scikit-learn.org/stable/modules/kernel_approximation.html#nystroem-method-for-kernel-approximation))
* ✔ 6.7.2. [**Noyau de fonction de base radiale**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#radial-basis-function-kernel)
([_Radial Basis Function Kernel_](https://scikit-learn.org/stable/modules/kernel_approximation.html#radial-basis-function-kernel))
* ✔ 6.7.3. [**Additif Chi Squared Kernel**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#)
([_Additive Chi Squared Kernel_](https://scikit-learn.org/stable/modules/kernel_approximation.html#additive-chi-squared-kernel))
* ✔ 6.7.4. [**Noyau au carré de chi asymétrique**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#)
([_Skewed Chi Squared Kernel_](https://scikit-learn.org/stable/modules/kernel_approximation.html#skewed-chi-squared-kernel))
* ✔ 6.7.5. [**Approximation du noyau polynomial via Tensor Sketch**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#)
([_Polynomial Kernel Approximation via Tensor Sketch_](https://scikit-learn.org/stable/modules/kernel_approximation.html#polynomial-kernel-approximation-via-tensor-sketch))
* ✔ 6.7.6. [**Détails mathématiques**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/6_7_kernel_approximation.ipynb#)
([_Mathematical Details_](https://scikit-learn.org/stable/modules/kernel_approximation.html#mathematical-details))

## <a id='nystroem-method-for-kernel-approximation'></a> 6.7.1. Méthode Nystroem pour l'approximation du noyau

La méthode de Nystroem, telle qu'elle est mise en œuvre dans [**`Nystroem`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.Nystroem.html), est une méthode générale d'approximation à faible rang des noyaux. Elle y parvient en sous-échantillonnant essentiellement les données sur lesquelles le noyau est évalué. Par défaut, [**`Nystroem`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.Nystroem.html) utilise le noyau `rbf`, mais il peut utiliser n'importe quelle fonction de noyau ou une matrice de noyau précalculée. Le nombre d'échantillons utilisés - qui est également la dimensionnalité des caractéristiques calculées - est donné par le paramètre `n_components`.

## <a id='radial-basis-function-kernel'></a> 6.7.2. Noyau de la fonction de base radiale (Radial Basis Function Kernel)

Le [**`RBFSampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html#sklearn.kernel_approximation.RBFSampler) construit une approximation de la fonction de base radiale du noyau, également connue sous le nom de _Random Kitchen Sinks_ [RR2007]. Cette transformation peut être utilisée pour modéliser explicitement une fonction de mapping de noyau, avant d'appliquer un algorithme linéaire, par exemple un SVM linéaire :

In [1]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
X = [[0, 0], [1, 1], [1, 0], [0, 1]]
y = [0, 0, 1, 1]
rbf_feature = RBFSampler(gamma=1, random_state=1)
X_features = rbf_feature.fit_transform(X)
clf = SGDClassifier(max_iter=5)
clf.fit(X_features, y)
# SGDClassifier(max_iter=5)
clf.score(X_features, y)
# 1.0

c:\Users\franc\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


1.0

La transformation repose sur une approximation de Monte Carlo des valeurs du noyau. La fonction `fit` effectue l'échantillonnage de Monte Carlo, tandis que la méthode `transform` effectue la transformation des données. En raison de la nature aléatoire du processus, les résultats peuvent varier entre différentes appels à la fonction `fit`.

La fonction `fit` prend deux arguments : `n_components`, qui est la dimensionnalité cible de la transformation des caractéristiques, et `gamma`, le paramètre du noyau RBF. Un `n_components` plus élevé donnera une meilleure approximation du noyau et produira des résultats plus similaires à ceux produits par un SVM à noyau. Notez que "l'ajustement" de la fonction de caractéristiques ne dépend pas réellement des données données à la fonction `fit`. Seule la dimensionnalité des données est utilisée. Les détails sur la méthode peuvent être trouvés dans [RR2007].

Pour une valeur donnée de `n_components`, [**`RBFSampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html) est souvent moins précis que [**`Nystroem`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.Nystroem.html). Cependant, le calcul de [**`RBFSampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html) est moins coûteux, ce qui rend l'utilisation d'espaces de caractéristiques plus grands plus efficace.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_kernel_approximation_002.png)

Comparaison d'un noyau RBF exact (à gauche) avec l'approximation (à droite)

### Exemples

####  [**Approximation explicite de la correspondance de caractéristiques pour les noyaux RBF**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/misc/plot_kernel_approximation.ipynb)<br/>([*Explicit feature map approximation for RBF kernels*](https://scikit-learn.org/stable/auto_examples/misc/plot_kernel_approximation.html))

## <a id='additive-chi-squared-kernel'></a> 6.7.3. Noyau chi carré additif (Additive Chi Squared Kernel)

Le noyau chi carré additif est un noyau sur les histogrammes, souvent utilisé en vision par ordinateur.

Le noyau chi carré additif utilisé ici est donné par

$$k(x, y) = \sum_i \frac{2x_iy_i}{x_i+y_i}$$

Ce n'est pas exactement la même chose que `sklearn.metrics.additive_chi2_kernel`. Les auteurs de [VZ2010] préfèrent la version ci-dessus car elle est toujours définie positive. Étant donné que le noyau est additif, il est possible de traiter toutes les composantes $x_i$ séparément pour le plongement. Cela permet d'échantillonner la transformée de Fourier à intervalles réguliers, au lieu d'utiliser une approximation par échantillonnage de Monte Carlo.

La classe [**`AdditiveChi2Sampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.AdditiveChi2Sampler.html) met en œuvre cet échantillonnage déterministe par composant. Chaque composant est échantillonné $n$ fois, ce qui donne $2n+1$ dimensions par dimension d'entrée (le multiple de deux provient de la partie réelle et de la partie complexe de la transformée de Fourier). Dans la littérature, $n$ est généralement choisi pour être 1 ou 2, transformant l'ensemble de données en une taille de `n_samples * 5 * n_features` (dans le cas de $n = 2$).

La fonctionnalité d'approximation de la correspondance de caractéristiques fournie par [**`AdditiveChi2Sampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.AdditiveChi2Sampler.html) peut être combinée avec l'approximation de la correspondance de caractéristiques fournie par [**`RBFSampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html) pour obtenir une approximation de la correspondance de caractéristiques pour le noyau exponentiel chi carré. Voir [VZ2010] pour plus de détails et [VVZ2010] pour la combinaison avec [**`RBFSampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html).

## <a id='skewed-chi-squared-kernel'></a> 6.7.4. Skewed Chi Squared Kernel (Skewed Chi Squared Kernel)

Le noyau chi carré asymétrique est donné par :

$$k(x,y) = \prod_i \frac{2\sqrt{x_i+c}\sqrt{y_i+c}}{x_i + y_i + 2c}$$

Il possède des propriétés similaires au noyau chi carré exponentiel souvent utilisé en vision par ordinateur, mais permet une approximation simple par échantillonnage de Monte Carlo de la correspondance de caractéristiques.

L'utilisation de [**`SkewedChi2Sampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.SkewedChi2Sampler.html) est similaire à celle décrite ci-dessus pour [**`RBFSampler`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.RBFSampler.html). La seule différence réside dans le paramètre libre appelé $\eta$. Pour une motivation de cette correspondance et les détails mathématiques, voir [LS2010].

## <a id='polynomial-kernel-approximation-via-tensor-sketch'></a> 6.7.5. Approximation du noyau polynomial via Tensor Sketch

Le [**noyau polynomial** (6.8.3)](https://scikit-learn.org/stable/modules/metrics.html#polynomial-kernel) est un type populaire de fonction de noyau donné par :

$$k(x, y) = (\gamma x^\top y +c_0)^d$$

où :

- $x$ et $y$ sont les vecteurs d'entrée,
- $d$ est le degré du noyau.

De manière intuitive, l'espace des caractéristiques du noyau polynomial de degré $d$ comprend tous les produits possibles de degré $d$ entre les caractéristiques d'entrée, ce qui permet aux algorithmes d'apprentissage utilisant ce noyau de tenir compte des interactions entre les caractéristiques.

La méthode TensorSketch [PP2013], mise en œuvre dans [**`PolynomialCountSketch`**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.PolynomialCountSketch.html), est une méthode évolutive, indépendante des données d'entrée, pour l'approximation du noyau polynomial. Elle est basée sur le concept de Count sketch [WIKICS] [CCF2002], une technique de réduction de dimensionnalité similaire au hachage de caractéristiques, qui utilise plusieurs fonctions de hachage indépendantes. TensorSketch obtient un Count Sketch du produit externe de deux vecteurs (ou d'un vecteur avec lui-même), qui peut être utilisé comme une approximation de l'espace des caractéristiques du noyau polynomial. En particulier, au lieu de calculer explicitement le produit externe, TensorSketch calcule le Count Sketch des vecteurs, puis utilise une multiplication polynomiale via la transformée de Fourier rapide pour calculer le Count Sketch de leur produit externe.

De manière pratique, la phase d'entraînement de TensorSketch consiste simplement à initialiser certaines variables aléatoires. Elle est donc indépendante des données d'entrée, c'est-à-dire qu'elle dépend uniquement du nombre de caractéristiques d'entrée, mais pas des valeurs des données. De plus, cette méthode peut transformer des échantillons en $\mathcal{O}(n_{\text{samples}}(n_{\text{features}} + n_{\text{components}} \log(n_{\text{components}})))$, où $n_{\text{components}}$ est la dimension de sortie souhaitée, déterminée par `n_components`.

### Exemples

#### [**Apprentissage extensible avec une approximation du noyau polynomial**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/6_7_kernel_approximation/plot_scalable_poly_kernels.ipynb)<br/>([_Scalable learning with polynomial kernel approximation_](https://scikit-learn.org/stable/auto_examples/kernel_approximation/plot_scalable_poly_kernels.html))

## <a id='mathematical-details'></a> 6.7.6. Détails mathématiques

Les méthodes de noyau telles que les machines à vecteurs de support (SVM) ou l'ACP noyautée reposent sur une propriété des espaces de Hilbert à noyau reproduisant. Pour toute fonction de noyau définie positive $k$ (appelée noyau de Mercer), il est garanti qu'il existe une correspondance dans un espace de Hilbert $\mathcal{H}$ telle que

$$k(x,y) = \langle \phi(x), \phi(y) \rangle$$

où $\langle \cdot, \cdot \rangle$ désigne le produit scalaire dans l'espace de Hilbert.

Si un algorithme, tel qu'une machine à vecteurs de support linéaire ou une ACP, ne dépend que du produit scalaire des points de données $x_i$, on peut utiliser la valeur de $k(x_i, x_j)$, ce qui correspond à appliquer l'algorithme aux points de données correspondants $\phi(x_i)$. L'avantage d'utiliser $k$ est que la correspondance $\phi$ n'a jamais besoin d'être calculée explicitement, ce qui permet d'avoir des caractéristiques arbitrairement grandes (voire infinies).

Un inconvénient des méthodes de noyau est qu'il peut être nécessaire de stocker de nombreuses valeurs de noyau $k(x_i, x_j)$ pendant l'optimisation. Si un classifieur noyauté est appliqué à de nouvelles données $y_j$, il est nécessaire de calculer $k(x_i, x_j)$ pour effectuer des prédictions, potentiellement pour de nombreux $x_i$ différents dans l'ensemble d'entraînement.

Les classes de ce sous-module permettent d'approximer la correspondance $\phi$, travaillant ainsi explicitement avec les représentations $\phi(x_i)$, ce qui évite d'appliquer le noyau ou de stocker les exemples d'entraînement.

### Références

🔬 [RR2007] (1,2) [“**Random features for large-scale kernel machines**](https://papers.nips.cc/paper/2007/hash/013a006f03dbc5392effeb8f18fda755-Abstract.html)[”](https://drive.google.com/file/d/1M3PZaLgkoCFmoQMUS8Wu9LOEPJdOc0OM/view?usp=drive_link) Rahimi, A. and Recht, B. - Advances in neural information processing 2007,

🔬 [LS2010] [“**Random Fourier approximations for skewed multiplicative histogram kernels**](https://www.researchgate.net/publication/221114584_Random_Fourier_Approximations_for_Skewed_Multiplicative_Histogram_Kernels)[”](https://drive.google.com/file/d/1niKSiwVYFC2O2666aVK6gNaFlgovgQJW/view?usp=drive_link) Li, F., Ionescu, C., and Sminchisescu, C. - Pattern Recognition, DAGM 2010, Lecture Notes in Computer Science.

🔬 [VZ2010] (1,2) [“**Efficient additive kernels via explicit feature maps**](https://www.robots.ox.ac.uk/~vgg/publications/2011/Vedaldi11/vedaldi11.pdf)[”](https://drive.google.com/file/d/1HYkw9kyMhEu8X49z97-Hl3yYwAyE4wJt/view?usp=drive_link) Vedaldi, A. and Zisserman, A. - Computer Vision and Pattern Recognition 2010

🔬 [VVZ2010] [“**Generalized RBF feature maps for Efficient Detection**](https://www.robots.ox.ac.uk/~vgg/publications/2010/Sreekanth10/sreekanth10.pdf)[”](https://drive.google.com/file/d/1W6SPhxHEWmPUMT-JvmC2MP4WPhK5UqW2/view?usp=drive_link) Vempati, S. and Vedaldi, A. and Zisserman, A. and Jawahar, CV - 2010

🔬 [PP2013] [“**Fast and scalable polynomial kernels via explicit feature maps**](https://dl.acm.org/doi/10.1145/2487575.2487591)[”](https://drive.google.com/file/d/1Hv80uCR8FVdlJX4u2Cqys8uQ6MHpIVLU/view?usp=drive_link) Pham, N., & Pagh, R. - 2013

🔬 [CCF2002] [“**Finding frequent items in data streams**](https://www.cs.princeton.edu/courses/archive/spring04/cos598B/bib/CharikarCF.pdf)[”](https://drive.google.com/file/d/1hHKakg5qubQJ0jSufITH68SE6odgh04J/view?usp=drive_link) Charikar, M., Chen, K., & Farach-Colton - 2002

🌐 [WIKICS] [“**Wikipedia: Count sketch**”](https://en.wikipedia.org/wiki/Count_sketch)